# Transverse Field Ising Model example

In the following example we will run the dmrg code for an Ising model for a single point in parameter space

## Initialize the

In [3]:
# import the MPS class from dmrg
from dmrg.MPS import MPS 

# specify the number of sites on the chain
L = 20 

# initialise the MPS for the indicated chain length
mps = MPS(20,mem='on')